In [ ]:
#
#
# 1 ~ 4
#
#

In [ ]:
# 5-1 기본 실습

# SimpleRNN (기본 순환 구조)
'''
숫자 시퀀스 예측
입력: [1, 2, 3] → 출력: 4
입력: [2, 3, 4] → 출력: 5
입력: [3, 4, 5] → 출력: 6
'''

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, Dense

# 시계열 데이터처럼 연속된 숫자를 기반으로 다음 값을 예측하는 문제

# 1. 데이터 준비
# 입력 시퀀스 (4개 샘플, 각 샘플은 길이 3인 시퀀스)
# 정답 (각 시퀀스의 다음 숫자)
def create_data(i):
  train_list = []
  val_list = []
  for j in range(1, i+1):
    train = [j, j+1, j+2]
    train_list.append(train)
    val = j+3
    val_list.append(val)
  train_list = np.array(train_list)
  val_list = np.array(val_list)
  return train_list, val_list
X, y = create_data(4)

# RNN 입력형태로 reshape: (샘플 수, 타임스텝 수, 특성 수)
# 숫자 하나씩 보므로 특성(feature)은 1
X = X.reshape((-1, 3, 1))

# 2. 모델 구성
model = Sequential()
# Input 레이어로 입력 형태를 명시 (3타임스텝, 1특성)
# SimpleRNN 레이어 (유닛 10개, 활성화함수: relu)
# Dense 출력층 (출력 노드 1개: 다음 숫자 1개 예측)
model.add(Input(shape=(X.shape[1:])))
model.add(SimpleRNN(32, activation='relu'))
model.add(Dense(1))

# 3. 모델 컴파일
# optimizer = 'adam': 최적화 알고리즘
# loss = 'mse': 회귀 문제이므로 평균제곱오차 사용
model.compile(
  optimizer = 'adam',
  loss = 'mse',
  metrics = ['mae']
)

# 4. 모델 학습
# epochs = 200: 반복 횟수
# verbose = 0: 진행 상황 출력 생략(0)
model.fit(
  X, y,
  epochs = 200,
  verbose = 1
)

# 5. 예측
# 예측 대상 시퀀스: [5, 6, 7]
# 샘플 수 차원 추가 필요
test_input = np.array([5, 6, 7]).reshape((1, 3, 1))

# 예측 수행
predicted = model.predict(test_input, verbose=0)
print("예측결과:", predicted[0][0])

In [ ]:
# 5-2 숫자 시퀀스 예측 예제 - SimpleRNN & LSTM & GRU

# 시퀀스 [1,2,3] → 4, [2,3,4] → 5 처럼 숫자 패턴을 학습해서 다음 수를 예측하는 간단한 RNN 모델을 만들고, 아래 세 가지 모델을 비교

# 모델 성능 해석 (MSE 기준)
# 모델 / 예측 정확도 / 특성 요약
# SimpleRNN / ex) 0.0152 / 가장 단순, 성능 무난
# LSTM / ex) 0.0041 / 장기 의존성 학습 잘함
# GRU / ex) 0.0029 / 성능 좋고 빠름

import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

# 1. 데이터 준비
# 입력 시퀀스 (4개 샘플, 각 샘플은 길이 3인 시퀀스)
# 정답 (각 시퀀스의 다음 숫자)
def create_data(i):
  train_list = []
  val_list = []
  for j in range(1, i+1):
    train = [j, j+1, j+2]
    train_list.append(train)
    val = j+3
    val_list.append(val)
  train_list = np.array(train_list)
  val_list = np.array(val_list)
  return train_list, val_list
X, y = create_data(6)

# 2. 입력 데이터 shape 정의
X = X.reshape((-1, 3, 1))

# 3. 모델 생성 함수
model_types = [SimpleRNN, LSTM, GRU]
results = []

def build_model(model_type):
  model = Sequential()
  model.add(Input(shape=(X.shape[1:])))
  model.add(model_type(32, activation='tanh'))
  model.add(Dense(1))
  
  model.compile(
  	optimizer = Adam(0.01),
  	loss = 'mse'
	)
  return model

# 4. 모델 학습, 예측, MSE 계산
for model_type in model_types:
  print(f"\n{model_type.__name__} 모델 학습 및 예측")
  model = build_model(model_type)
  
  model.fit(
		X, y,
		epochs = 100,
		verbose = 0
	)
  preds = model.predict(X)
  mse = mean_squared_error(y, preds)
  
  for i in range(len(X)):
    results.append({
      "모델": model_type.__name__,
      "입력 시퀀스": X[i].flatten().tolist(),
      "실제값": y[i],
      "예측값": round(preds[i][0], 3),
      "MSE": round(mse, 4)
		})
    
# 5. 결과 정리 및 출력
df = pd.DataFrame(results)
display(df)

In [ ]:
# 6. 문장 예측 실습

# "hello" → 다음 글자 예측하기 (3가지 RNN 비교 예제)

import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import mean_squared_error

# 문자 집합 정의
text = "hello"
chars = sorted(list(set(text)))

char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for i, c in enumerate(chars)}
print("문자-인덱스 딕셔너리 생성")
print(char_to_index)
print(index_to_char)

# 문자 → 숫자 변환
X_data = [char_to_index[c] for c in text[:-1]]
y_data = [char_to_index[c] for c in text[1:]]
print("\n딕셔너리를 이용한 문자 인덱스화")
print(X_data)
print(y_data)

# One-Hot Encoding
X = to_categorical(X_data, num_classes=len(chars))
y = to_categorical(y_data, num_classes=len(chars))

# RNN 입력 형식으로 변환: (samples, timesteps, features)
X = X.reshape(1, 4, len(chars))
y = y.reshape(1, 4, len(chars))

# 결과 저장 딕셔너리
results = {}

# 공통 함수: 모델 생성, 학습 예측, 평가
def train_and_predict(model_name, RNNlayer):
  model = Sequential()
  model.add(Input(shape=(X.shape[1:])))
  model.add(RNNlayer(10, return_sequences=True))
  model.add(Dense(len(chars), activation='softmax'))
  
  model.compile(
		optimizer = 'adam',
		loss = 'categorical_crossentropy'
	)
  
  # 학습
  model.fit(
		X, y,
		epochs = 500,
		verbose = 0
	)
  
  # 예측
  pred = model.predict(X, verbose=0)
  
  pred_i = np.argmax(pred, axis=2)
  true_i = np.argmax(y, axis=2)
  
  pred_chars = [index_to_char[i] for i in pred_i[0]]
  true_chars = [index_to_char[i] for i in true_i[0]]
  
  mse = mean_squared_error(true_i[0], pred_i[0])
  
  results[model_name] = {
		"예측값": pred_chars,
		"실제값": true_chars,
		"MSE": round(mse, 4)
	}
  
# 각 RNN 방식별 실행
train_and_predict("SimpleRNN", SimpleRNN)
train_and_predict("LSTM", LSTM)
train_and_predict("GRU", GRU)
  
# 결과 출력
df = pd.DataFrame(results).T
display(df)

In [ ]:
# 7-1 RNN 을 이용한 문장 생성(예측) 예제

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Input
from tensorflow.keras.utils import to_categorical

# 1. 데이터 전처리
# 학습할 문장
text = "hello world"

# 고유 문자 집합 만들기
chars = sorted(list(set(text)))
char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for i, c in enumerate(chars)}
print("문자-인덱스 딕셔너리 생성")
print(char_to_index)
print(index_to_char)
print(f"문장 길이: {len(text)}\n")

# 시퀀스 길이 설정 (3글자를 보고 1글자 예측)
seq_length = 3

# 학습 데이터(X), 라벨(y) 생성
X_data = []
y_data = []
for i in range(len(text) - seq_length):
  seq_in = text[i:i+seq_length] # ex) "hel"
  seq_out = text[i+seq_length] # ex) "l"
  X_data.append([char_to_index[c] for c in seq_in])
  y_data.append(char_to_index[seq_out])
print(f"X: {X_data}")
print(f"y: {y_data}")

# One-Hot Encoding
X = to_categorical(X_data, num_classes=len(chars))
y = to_categorical(y_data, num_classes=len(chars))

# 2. 모델 구성 및 학습
model = Sequential()
model.add(Input(shape=(seq_length, len(chars))))
model.add(SimpleRNN(128)) # activation 기본값 tanh
model.add(Dense(len(chars), activation='softmax'))

model.compile(
  optimizer = 'adam',
  loss = 'categorical_crossentropy',
  metrics = ['accuracy']
)

# model.summary
model.summary()

model.fit(
  X, y,
  epochs = 500,
  verbose = 0
)

# 3. 문장 생성 및 함수
def generate_text(seed_text, n_chars):
  result = seed_text
  for _ in range(n_chars):
    input_idx = [char_to_index[c] for c in seed_text]
    input_seq = to_categorical([input_idx], num_classes=len(chars))
    
    # 예측
    prediction = model.predict(input_seq, verbose=0)
    next_idx = np.argmax(prediction[0])
    next_char = index_to_char[next_idx]
    
    # 결과 추가
    result += next_char
    print(f"문자 생성중...: {result}")
    
    # 다음 입력을 위한 시드 업데이트
    seed_text = result[-seq_length:]
  return result

# 실행 예시
# "hel" 값으로 20글자 생성
print(f"\n최종 문장: {generate_text("hel", 20)}")

In [ ]:
# 7-2 LSTM 을 이용한 문장 생성(예측) 예제

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.utils import to_categorical

# 학습할 문장
text = "hello world"

# 고유 문자 집합 만들기
chars = sorted(list(set(text)))
char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for i, c in enumerate(chars)}

# 시퀀스 길이 설정 (3글자를 보고 다음 1글자 예측)
seq_length = 3

# 입력 시퀀스(X), 다음 문자(y) 만들기
X_data = []
y_data = []
for i in range(len(text) - seq_length):
  seq_in = text[i:i+seq_length]
  seq_out = text[i+seq_length]
  X_data.append([char_to_index[c] for c in seq_in])
  y_data.append(char_to_index[seq_out])
  
# One-Hot Encoding
X = to_categorical(X_data, num_classes=len(chars))
y = to_categorical(y_data, num_classes=len(chars))

# 모델 구성 및 학습
model = Sequential()
model.add(Input(shape=(seq_length, len(chars))))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
  optimizer = 'adam',
  loss = 'categorical_crossentropy',
  metrics = ['accuracy']
)

# model.summary
model.summary()

model.fit(
  X, y,
  epochs = 500,
  verbose = 0
)

# 3. 문장 생성 및 함수
def generate_text(seed_text, n_chars):
  result = seed_text
  for _ in range(n_chars):
    input_idx = [char_to_index[c] for c in seed_text]
    input_seq = to_categorical([input_idx], num_classes=len(chars))
    
    # 예측
    prediction = model.predict(input_seq, verbose=0)
    next_idx = np.argmax(prediction[0])
    next_char = index_to_char[next_idx]
    
    # 결과 추가
    result += next_char
    print(f"문자 생성중...: {result}")
    
    # 다음 입력을 위한 시드 업데이트
    seed_text = result[-seq_length:]
  return result

# 실행 예시
# "hel" 값으로 20글자 생성
print(f"\n최종 문장: {generate_text("hel", 20)}")

In [ ]:
# 7-3 GRU 을 이용한 문장 생성(예측) 예제

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Input
from tensorflow.keras.utils import to_categorical

# 학습할 문장
text = "hello world"

# 고유 문자 집합 만들기
chars = sorted(list(set(text)))
char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for i, c in enumerate(chars)}

# 시퀀스 길이 설정 (3글자를 보고 다음 1글자 예측)
seq_length = 3

# 입력 시퀀스(X), 다음 문자(y) 만들기
X_data = []
y_data = []
for i in range(len(text) - seq_length):
  seq_in = text[i:i+seq_length]
  seq_out = text[i+seq_length]
  X_data.append([char_to_index[c] for c in seq_in])
  y_data.append(char_to_index[seq_out])
  
# One-Hot Encoding
X = to_categorical(X_data, num_classes=len(chars))
y = to_categorical(y_data, num_classes=len(chars))

# 모델 구성 및 학습
model = Sequential()
model.add(Input(shape=(seq_length, len(chars))))
model.add(GRU(128))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
  optimizer = 'adam',
  loss = 'categorical_crossentropy',
  metrics = ['accuracy']
)

# model.summary
model.summary()

model.fit(
  X, y,
  epochs = 500,
  verbose = 0
)

# 3. 문장 생성 및 함수
def generate_text(seed_text, n_chars):
  result = seed_text
  for _ in range(n_chars):
    input_idx = [char_to_index[c] for c in seed_text]
    input_seq = to_categorical([input_idx], num_classes=len(chars))
    
    # 예측
    prediction = model.predict(input_seq, verbose=0)
    next_idx = np.argmax(prediction[0])
    next_char = index_to_char[next_idx]
    
    # 결과 추가
    result += next_char
    print(f"문자 생성중...: {result}")
    
    # 다음 입력을 위한 시드 업데이트
    seed_text = result[-seq_length:]
  return result

# 실행 예시
# "hel" 값으로 20글자 생성
print(f"\n최종 문장: {generate_text("hel", 20)}")

In [ ]:
# 8. 한글 긴 문장이나 짧은 문서 텍스트를 RNN & LSTM & GRU 기반으로 학습하는 방법

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, SimpleRNN, LSTM, GRU
from tensorflow.keras.utils import to_categorical

text = "인공지능은 인간의 지능을 모방하여 다양한 문제를 해결하는 기술입니다. 자연어 처리, 이미지 분석, 추천 시스템 등에 활용됩니다"

chars = sorted(list(set(text)))
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for i, c in enumerate(chars)}

seq_length = 10

X_data = []
y_data = []
for i in range(len(text) - seq_length):
  seq_in = text[i:i+seq_length]
  seq_out = text[i+seq_length]
  X_data.append([char_to_idx[c] for c in seq_in])
  y_data.append(char_to_idx[seq_out])

X = to_categorical(X_data, num_classes=len(chars))
y = to_categorical(y_data, num_classes=len(chars))

RNN_types = [SimpleRNN, LSTM, GRU]

def build_model(RNN_list):
  model = Sequential()
  model.add(Input(shape=(seq_length, len(chars))))
  model.add(RNN_list(512))
  model.add(Dense(len(chars), activation='softmax'))
  
  model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy'
	)
  return model

for RNN_type in RNN_types:
  print(f"\n{RNN_type.__name__} 모델 학습 및 예측")
  model = build_model(RNN_type)
  model.summary()
  
  model.fit(
    X, y,
    epochs = 100,
    verbose = 0
	)
  
  def generate_text(seed_text, n_chars):
    result = seed_text
    for _ in range(n_chars):
      input_idx = [char_to_idx[c] for c in seed_text]
      input_seq = to_categorical([input_idx], num_classes=len(chars))
      
      
      pred = model.predict(input_seq, verbose=0)
      next_idx = np.argmax(pred[0])
      next_char = idx_to_char[next_idx]
      
      result += next_char
      # print(f"{RNN_type.__name__} 모델 문자 생성중: {result}")
      
      seed_text = result[-seq_length:]
    return result
  
  print(f"\n최종 문장: {generate_text("인공지능은", 50)}")